# Extract Spending Data by Fiscal Year

In [23]:
import requests
import xml.etree.ElementTree as ET
import time
import pandas as pd
from pathlib import Path
import xml.dom.minidom
import math
from tqdm import tqdm
import pyarrow as pa
import pyarrow.parquet as pq

ModuleNotFoundError: No module named 'tqdm.notebooks'

In [2]:
API_URL = "https://www.checkbooknyc.com/api"
TEMP_XML_FILE = Path("tmp_latest_response.xml")  # same temp file each time
TIMEOUT = (300, 300) # connect_timeout, read_timeout for requests
OUTPUT_DIR = Path("checkbook_data")
OUTPUT_DIR.mkdir(exist_ok=True)
BATCH_SIZE = 20000  # API's retrieval limit on records per request

In [3]:
# session set up: bypass incapsula bot protection using session mgmt + browser headers
session = requests.Session()
HEADERS = {
    'content-type': 'application/xml',
    'user-agent': 'mozilla/5.0 (windows nt 10.0; win64; x64) applewebkit/537.36'
}

session.headers.update(HEADERS)

# * visit main site initially *
response = session.get('https://www.checkbooknyc.com/', timeout=TIMEOUT)
print(f"main site status: {response.status_code}")
time.sleep(2)

session.headers.update({
    'content-type': 'application/xml',
    'referer': 'https://www.checkbooknyc.com/',
    'origin': 'https://www.checkbooknyc.com'
})

main site status: 200


## Helpers

In [4]:
def fetch_xml(payload: str, verbose: bool = False) -> ET.Element:
    response = session.post(API_URL, data=payload, headers=HEADERS, timeout=TIMEOUT)
    if verbose: 
        print(response.text[:500])
    response.raise_for_status()

    if verbose:
        xml_str = response.content.decode('utf-8')
        print(xml_str)
        
    return ET.fromstring(response.content)

def get_record_count(xml_root: ET.Element) -> int:
    count_tag = xml_root.find(".//result_records/record_count")
    return int(count_tag.text) if count_tag is not None else 0



In [5]:
y = "2025"
y[2:]

'25'

## Initial Metadata Request: Determining Pagination

In [6]:
fiscal_year = 2025

In [7]:
seed_request=f"""
<request>
  <type_of_data>Spending</type_of_data>
  <records_from>1</records_from>
  <max_records>1</max_records>
  <search_criteria>
   <criteria>
      <name>fiscal_year</name>
      <type>value</type>
      <value>{fiscal_year}</value>
    </criteria>
  </search_criteria>
  <response_columns/>
</request>
"""

In [8]:
seed_root = fetch_xml(seed_request,verbose=True)

<?xml version="1.0"?>
<response>
  <status>
    <result>success</result>
  </status>
  <request_criteria>
    <request>
      <type_of_data>Spending</type_of_data>
      <records_from>1</records_from>
      <max_records>1</max_records>
      <search_criteria>
        <criteria>
          <name>fiscal_year</name>
          <type>value</type>
          <value>2025</value>
        </criteria>
      </search_criteria>
      <response_columns/>
    </request>
  </request_criteria>
  <result_records>

<?xml version="1.0"?>
<response>
  <status>
    <result>success</result>
  </status>
  <request_criteria>
    <request>
      <type_of_data>Spending</type_of_data>
      <records_from>1</records_from>
      <max_records>1</max_records>
      <search_criteria>
        <criteria>
          <name>fiscal_year</name>
          <type>value</type>
          <value>2025</value>
        </criteria>
      </search_criteria>
      <response_columns/>
    </request>
  </request_criteria>
  <result_records>

In [9]:
retrievable_records = get_record_count(seed_root)
batches_needed = math.ceil(retrievable_records / BATCH_SIZE)

print(f"""
Seed Request Results for FY {fiscal_year}
=========================================
Total retrievable records: {retrievable_records}
Batches needed: {batches_needed}
""")


Seed Request Results for FY 2025
Total retrievable records: 3157155
Batches needed: 158



## Start Paginated Fetch-to-Appends

## Fetch and Save FY Spending

### Function and Helpers

In [19]:
tqdm._instances.clear()

def save_parquet_append(df: pd.DataFrame, out_file: Path) -> int:
    out_file = Path(out_file)
    if out_file.exists():
        try:
            pa.unregister_extension_type("pandas.period")
        except KeyError:
            pass
        existing_df = pd.read_parquet(out_file, engine="pyarrow")
        original_count = len(existing_df)
        combined = pd.concat([existing_df, df], ignore_index=True)
        deduped = combined.drop_duplicates()
        
        # fix: calculate actual new records added
        final_count = len(deduped)
        new_records_attempted = len(df)
        
        # net change can be negative, but "added" should be non-negative
        net_change = final_count - original_count
        actually_added = max(0, net_change)  # never negative
        
        deduped.to_parquet(out_file, engine="pyarrow", index=False)
        return actually_added
    else:
        df.to_parquet(out_file, engine="pyarrow", index=False)
        return len(df)

        
def parse_transactions(xml_root: ET.Element) -> pd.DataFrame:
    """convert <transaction> elements into dataframe rows"""
    records = []
    for txn in xml_root.findall(".//transaction"):
        row = {child.tag: (child.text or "").strip() for child in txn}
        records.append(row)
    return pd.DataFrame(records)

def download_all_spending(xml_template: str, year: int, total_available: int, verbose: bool = True):
    """download all available spending records for given year"""
    from tqdm import tqdm
    import time
    
    out_file = OUTPUT_DIR / f"spending_{year}.parquet"
    
    # check current progress
    if out_file.exists():
        existing_df = pd.read_parquet(out_file)
        records_downloaded = len(existing_df)
    else:
        records_downloaded = 0
    
    records_remaining = total_available - records_downloaded
    
    if records_remaining <= 0:
        if verbose:
            print(f"download already complete: {records_downloaded:,} records")
        return
    
    if verbose:
        print(f"downloading {records_remaining:,} remaining records (total: {total_available:,})")
    
    with tqdm(total=records_remaining, desc=f"FY{year}", unit="records", ncols=100) as pbar:
        while records_remaining > 0:
            # calculate batch size for this request
            batch_size = min(BATCH_SIZE, records_remaining)
            
            try:
                # download one batch
                records_saved = download_spending_atomic(xml_template, year, batch_size, verbose=False)
                
                # update progress
                records_remaining -= records_saved
                pbar.update(records_saved)
                
                if verbose:
                    total_now = total_available - records_remaining
                    tqdm.write(f"batch complete: {records_saved:,} saved, {total_now:,} total, {records_remaining:,} remaining")
                
                # rate limiting - api requires 1 request per second
                time.sleep(1.1)
                
            except Exception as e:
                tqdm.write(f"batch failed: {e}")
                tqdm.write("retrying in 5 seconds...")
                time.sleep(5)
                # continue loop without updating counters - will retry same batch
    
    if verbose:
        final_df = pd.read_parquet(out_file)
        print(f"download complete: {len(final_df):,} total records saved")

# usage:
xml_template = """<request>
  <type_of_data>Spending</type_of_data>
  <records_from>{records_from}</records_from>
  <max_records>{max_records}</max_records>
  <search_criteria>
   <criteria>
      <name>fiscal_year</name>
      <type>value</type>
      <value>{fiscal_year}</value>
    </criteria>
  </search_criteria>
  <response_columns>
    <column>agency</column>
    <column>payee_name</column>
    <column>check_amount</column>
    <column>issue_date</column>
    <column>document_id</column>
    <column>spending_category</column>
    <column>department</column>
    <column>fiscal_year</column>
  </response_columns>
</request>"""

In [20]:
# download all 3+ million records
download_all_spending(xml_template, 2025, retrievable_records)

downloading 3,117,155 remaining records (total: 3,157,155)


FY2025:   1%|▎                                       | 20000/3117155 [00:04<11:48, 4372.98records/s]

batch complete: 20,000 saved, 60,000 total, 3,097,155 remaining


FY2025:   1%|▌                                       | 40000/3117155 [00:10<13:49, 3710.48records/s]

batch complete: 20,000 saved, 80,000 total, 3,077,155 remaining


...

batch complete: 20,000 saved, 3,140,000 total, 17,155 remaining


FY2025: 100%|███████████████████████████████████████| 3117155/3117155 [27:03<00:00, 932.96records/s]

batch complete: 17,155 saved, 3,157,155 total, 0 remaining


FY2025: 100%|██████████████████████████████████████| 3117155/3117155 [27:04<00:00, 1918.78records/s]


download complete: 3,157,155 total records saved


### Function Calls

## Peek in Parquet Files

In [ ]:
# file_path = "checkbook_data/spending_2025.parquet"
file_path = "checkbook_data/duplicates_spending_2025.parquet"


# read the Parquet file into a DataFrame
df = pd.read_parquet(file_path, engine="pyarrow") 

# look at the first few rows
print(df.head())

# see general info
print(df.info())

# get column names
print(df.columns)

In [22]:
# check what actually got saved
out_file = OUTPUT_DIR / f"spending_{fiscal_year}.parquet"
if out_file.exists():
    df_check = pd.read_parquet(out_file)
    print(f"file exists: {len(df_check)} records")
else:
    print("no file created")
    
# check if xml was saved
xml_files = list(OUTPUT_DIR.glob("raw_FY*.xml"))
print(f"xml files: {len(xml_files)}")

file exists: 3157155 records
xml files: 6


###  API Notes

https://www.checkbooknyc.com/spending-api

- retrieval limit: 20K records per call
- FY25: 3.2M transactions 
- to retrieve all: stagger `records_from` 

3,157,063 transactions <- as of 8/22 from data feeds https://www.checkbooknyc.com/data-feeds 

##### XML request:
- global params: `type_of_data`, `records_from`, `max_records`
- filters: put in `<search_criteria>`
- requested columns: put in `<response_columns>`

##### XML response: 
- `<search_criteria>`: echoes request xml
- `<result_records>`: current result batch based on `records_from` and `max_records`
- `<record_count>`: TOTAL rows
- `<messages>`: errors + status info
- `<status>`: success/failure of request

##### prossibly outdated thread on rate limits
https://groups.google.com/g/checkbooknyc/c/hgU1niDG00Y?pli=1

## technical notes

### api integration:
- bypass incapsula bot protection using session mgmt + browser headers
- learn xml format structure requirements of checkbooknyc api
- debug XML templating issues betewen f-strings vs `.format()`

### pagination / data integrity:
- **problem**: deduplication approach failed - legitimate different records marked as duplicates
- **root cause**: overlapping requests (requesting records 1-20000 when file had 1-19815)
- **solution**: range tracking with atomic saves eliminates deduplication need

### download architecture:
- resumable downloads using file length to calculate next offset
- atomic batch operations ensure data consistency if interrupted
- complying with rate limiting + error handling with auto retry
- progress tracking for slo downloads (e.g., full FY)

### key insights
- solve api pagination problem at the request level (proper offsets) instead of data level (deduplication).
- separate concerns: overlapping requests vs. data consistency
- well-structured batch system with proper offsets -> remarkably less data mucking / data handling complexity